In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
import os
import pickle
import re

In [2]:
# comments have already been downloaded (using curl)
# and are in html/post_pages/
# there are about 400,000 of them

# we want to get:
# game name
# game id if available
# post author name
# post author id
# post content
# post length
# timestamp
# anything else???

post_dir = 'html/post_pages'

In [3]:
post_list = [x for x in os.listdir(post_dir) if x[-5:] == ".html"]
post_list.sort()
len(post_list)

89869

In [38]:
pf_df = pd.DataFrame(columns=['user name', 'time stamp', 'span count', 'pee count', 'trail anchor', 'title'])

for file in post_list[2100:2200]:
    # get post id from url
    pid_l = re.findall(r'\d+', file)
    pid = int(pid_l[0])
    
    # make a soup
    this_post = 'html/post_pages/' + file
    with open (this_post, 'r', encoding="ISO-8859-1") as fh:
        rt = fh.read()
    soup = bs(rt, "lxml")
    
    # get title
    title_txt = soup.title.string
    
    # get span count
    all_spans = soup.find_all('span')
    all_pees = soup.find_all('p')
    trail_anchor = soup.find_all('a', class_="trail")
    span_count = len(all_spans)
    pee_count = len(all_pees)
    
    if len(trail_anchor) > 0:
        trail_url = trail_anchor[0]['href']
    else:
        trail_url = '';
    
    if span_count > 6:
        user_name = all_spans[1].string
        stamp = all_spans[2]['title']
    
    pf_df.loc[pid] = [user_name, stamp, span_count, pee_count, trail_url, title_txt]
    

In [40]:
pf_df.sort_index(inplace=True)
pf_df.T

,1143,1144,1145,1146,1147,1148,1149,11428,11429,11430,...,114714,114754,114784,114821,114855,114873,114894,114923,114941,114962
user name,AaronShurmon,Lavicc,Clemo,xAD,barodapride,Absolute Mayhem,AaronShurmon,Phoninskis,nikolam,tesselode,...,Mokais,bubbabubba519,The bridy,Oneiric Game Development,Shiirn,Dasius,k12th,scitydreamer,RALPHGAMEZ,nomercy093
time stamp,2015-10-26 02:22:20,2015-10-26 04:09:08,2015-10-26 04:09:54,2015-10-26 04:51:45,2015-10-26 06:36:08,2015-10-26 07:29:49,2015-10-26 08:16:33,2016-01-17 23:16:02,2016-01-17 23:25:47,2016-01-17 23:52:21,...,2017-02-09 23:53:46,2017-02-10 01:19:10,2017-02-10 02:44:26,2017-02-10 04:07:37,2017-02-10 06:31:14,2017-02-10 08:10:25,2017-02-10 09:38:33,2016-01-18 06:39:21,2017-02-10 11:47:48,2017-02-10 13:18:16
span count,16,13,10,42,12,10,9,11,9,14,...,13,9,11,9,12,23,17,6,9,11
pee count,4,3,3,41,2,1,1,1,2,4,...,2,1,1,1,4,4,4,0,3,1
trail anchor,https://loficacti.itch.io/friendly-bomb,https://loficacti.itch.io/friendly-bomb,https://oneaperture.itch.io/drone-ikuze,https://greenheartgames.itch.io/game-dev-tycoon,https://themissinglint.itch.io/jam-or-be-jam,https://fallendragonfly.itch.io/6-shots-2-guns,https://loficacti.itch.io/friendly-bomb,/jam/my-first-game-jam,/jam/libgdxjam,,...,https://oneiricgamedev.itch.io/blacklight,https://raft.itch.io/raft,https://dasius.itch.io/my-little-blacksmith-shop,https://oneiricgamedev.itch.io/blacklight,https://ebihime.itch.io/sweetest-monster,https://dasius.itch.io/my-little-blacksmith-shop,https://grrozny.itch.io/minor-fall-major-lift,,https://ralphgamez.itch.io/project-sh00tr,https://dasius.itch.io/my-little-blacksmith-shop
title,Post by AaronShurmon in Friendly Bomb (JAM Pro...,Post by Lavicc in Friendly Bomb (JAM Prototype...,Post by Clemo in Drone Ikuze comments - itch.io,Post by xAD in Game Dev Tycoon comments - itch.io,Post by barodapride in Jam or Be Jammed commen...,"Post by Absolute Mayhem in 6 Shots, 2 Guns com...",Post by AaronShurmon in Friendly Bomb (JAM Pro...,Post by Phoninskis in [DevLog] FOREST PASSAGE ...,Post by nikolam in NikolaM's DevLog - itch.io,Post by tesselode in Spellrazor - itch.io,...,Post by Mokais in Blacklight comments - itch.io,Post by bubbabubba519 in Building hammer contr...,Post by The bridy in Bug Reports - itch.io,Post by Oneiric Game Development in Blacklight...,Post by Shiirn in Sweetest Monster comments - ...,Post by Dasius in Bug Reports - itch.io,Post by k12th in Minor Fall Major Lift comment...,itch.io,Post by RALPHGAMEZ in Suggestions - itch.io,Post by nomercy093 in Complaint Box - itch.io


In [ ]:
def info_from_game_post_file(file):
    # get post id from url
    pid_l = re.findall(r'\d+', file)
    pid = int(pid_l[0])
    
    # div id changes for every file
    div_id = 'post-' + pid
    
    # print(did)
    with open (file, 'r', encoding="ISO-8859-1") as fh:
        rt = fh.read()
    soup = bs(rt, "lxml")
    
    # print(rt)
    print(len(rt))
    print(file)
    # check to see if it's a post page and not a 404 or whatevs
    title_txt = soup.title.string
    print(title_txt[:7])
    if title_txt[:7] == 'Post by':
        
        # dclass = "post_content"
        # div = soup.find('div', id=div_id)
    
    # dtag = soup.div
    # sdiv = dtag.attrs
    
    # ph_id = "post_header"
    
    # ph_div = soup.find('div', ph_id)
    # ph_div_cs = ph_div.findChildren()
    # ph_div_cs = ph_div.find_all("span")
        ph_div_cs = soup.find_all('span')
    # for child in ph_div_cs:
    #     print(child)
        user_span = str(ph_div_cs[1])
        stamp_span = str(ph_div_cs[2])
        user_name = ph_div_cs[1].getText()
        user_id = ph_div_cs[1]('a')[0]['href']
        stamp = stamp_span[31:-46]
        game_url = soup.findAll('h2')[0]('a')[0]['href']
        rettup = (pid, user_id, user_name, game_url, stamp)
    
        return rettup

In [ ]:
this_post = 'html/post_pages/' + post_list[111]
# print(this_post)
urg = info_from_game_post_file(this_post)
print(urg)

In [ ]:
len(post_list)

In [ ]:
post_list.index('117509.html')

In [ ]:
post_list[2283]

In [ ]:
post_tup_list = list()
for post_file in post_list:
    this_file = 'html/post_pages/' + post_file
    post_tup_list.append(info_from_game_post_file(this_file))

In [ ]:
post_tup_list

In [ ]:
len(post_tup_list)